In [6]:
import math
import random

import pandas as pd
from pysorteddict import SortedDict


def setup(sorted_dict_len: int, seed: float = math.pi) -> SortedDict:
    random.seed(seed)
    d = SortedDict()
    for _ in range(sorted_dict_len):
        d[random.random()] = random.random()
    return d


sorted_dict_lens = [100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]

# `__contains__`

In [ ]:
df = pd.DataFrame(index=sorted_dict_lens, columns=["0.00 in d", "0.33 in d", "0.66 in d", "1.00 in d"])
for i, sorted_dict_len in enumerate(sorted_dict_lens):
    print(f"{sorted_dict_len=}")
    d = setup(sorted_dict_len)
    for j, key in enumerate([0.00, 0.33, 0.66, 1.00]):
        df.iat[i, j] = %timeit -o key in d
df

sorted_dict_len=100
37 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
44.7 ns ± 0.489 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
43.5 ns ± 0.317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
40.4 ns ± 0.538 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
32 ns ± 0.483 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
sorted_dict_len=1000
48.8 ns ± 0.383 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
62 ns ± 0.346 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
55.6 ns ± 0.394 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
56.6 ns ± 0.384 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
61.9 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
sorted_dict_len=10000
63.4 ns ± 0.556 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
78.9 ns ± 0.455 ns per loop (mean ± std. dev. of 7 runs, 10,000,

# `SortedDictKeys.__iter__`

In [ ]:
for sorted_dict_len in sorted_dict_lens:
    print(f"{sorted_dict_len=}")
    d = setup(sorted_dict_len)
    k = d.keys()
    %timeit for _ in k: pass

# `SortedDict.__setitem__`